![ViewTree.png](ViewTree.png)

## 实现类型
- 继承于 View
- 继承自现有控件(如 TextView)
- 继承于 ViewGroup(FrameLayout)

### 重点
- View 的测量与布局
- View 的绘制
- 触摸事件
- 动画

---

## 自定义 View
- 灵活、复杂
- onMeasure、onDraw

### 步骤
- 继承自 View 创建自定义控件
- 如需要自定义属性，在 values/attrs.xml 中定义属性集
- 在 xml 中引入命名空间，设置属性
- 自定义 view 中读取属性，初始化视图
- 测量视图大小
- 绘制视图内容

---

## 尺寸测量
- 创建视图时调用跟视图的 measure、layout、draw 三个函数
- 对于非 ViewGroup 类型来说，layout 步骤是不需要的
- 绘制流程：ViewRoot.performTraversals() -> View.measure(widthMeasureSpec, heightMeasureSpec)

![SpecMode类型.png](SpecMode类型.png)

- MeasureSpec:ViewRootImpl 中创建的
  - rootDimension = MATCH_PARENT -> MeasureSpec = EXACTLY、specSize = windowSize(满屏)
  - rootDimension = WRAP_CONTENT -> MeasureSpec = AT_MOST、specSize = windowSize(满屏)

```
private static int getRootMeasureSpec(int windowSize, int rootDimension) {
    int measureSpec;
    switch (rootDimension) {
        case ViewGroup.LayoutParams.MATCH_PARENT:
            // Window can't resize. Force root view to be windowSize.
            measureSpec = MeasureSpec.makeMeasureSpec(windowSize, MeasureSpec.EXACTLY);
            break;
        case ViewGroup.LayoutParams.WRAP_CONTENT:
            // Window can resize. Set max size for root view.
            measureSpec = MeasureSpec.makeMeasureSpec(windowSize, MeasureSpec.AT_MOST);
            break;
        default:
            // Window wants to be an exact size. Force root view to be that size.
            measureSpec = MeasureSpec.makeMeasureSpec(rootDimension, MeasureSpec.EXACTLY);
            break;
    }
    return measureSpec;
}
```

- 自定义 View 的 onMeasure 方法
  - 最终调用 setMeasuredDimension 函数设置该 view 的尺寸

```
private int measureWidth(int mode, int width) {
    switch (mode) {
        case MeasureSpec.UNSPECIFIED:
        case MeasureSpec.AT_MOST:
            // 使用图片宽度(WRAP_CONTENT、不指定)
            break;
        case MeasureSpec.EXACTLY:
            // 使用指定宽度(MATCH_PARENT、具体值)
            mWidth = width;
            break;
    }
    return mWidth;
}
```

---

## 简易 demo
- 根据组件设置的图片 drawable，获取大小
  - 1. 用户设置了具体值，比如 MATCH_PARENT、180dp，则对应的 Mode 为 EXACTLY
  - 2. 用户未设置具体值，比如 WRAP_CONTENT，则对应的 Mode 为 AT_MOST
  - 3. 其他，一般不太实用 UNSPECIFIED（一般用于）

### SimpleImageView 类
- 根据图片大小以及组件的属性设置，来确定组件的大小，下面已 width 来举例
  - MATCH_PARENT: 父容器给到的组件大小值，这个 case 是撑满全屏的
  - WRAP_CONTENT: 图片大小和父容器给到的组件大小值比较，选择小的一个值作为最终尺寸
  - 180dp(举例): 父容器给到的组件大小值，这个 case 是 180dp

```
public class SimpleImageView extends View {
    private static final String TAG = SimpleImageView.class.getSimpleName();

    private Drawable mDrawable;
    private Paint mPaint;
    private Bitmap mBitmap;
    private int mWidth;
    private int mHeight;

    public SimpleImageView(Context context, @Nullable AttributeSet attrs) {
        super(context, attrs);
        initAttrs(attrs);
        mPaint = new Paint(Paint.ANTI_ALIAS_FLAG);
    }

    private void initAttrs(AttributeSet attrs) {
        if (attrs != null) {
            TypedArray typedArray = null;
            try {
                typedArray = getContext().obtainStyledAttributes(attrs, R.styleable.SimpleImageView);
                mDrawable = typedArray.getDrawable(R.styleable.SimpleImageView_src);
                measureDrawable();
            } finally {
                if (typedArray != null) {
                    typedArray.recycle();
                }
            }
        }
    }

    private void measureDrawable() {
        if (mDrawable == null) {
            throw new RuntimeException("drawable can not null!");
        }
        mWidth = mDrawable.getIntrinsicWidth();
        mHeight = mDrawable.getIntrinsicHeight();
    }

    @Override
    protected void onMeasure(int widthMeasureSpec, int heightMeasureSpec) {
        Log.d(TAG, "before width,height=" + (mWidth + "," + mHeight));
        mWidth = resolveSize(mWidth, widthMeasureSpec);
        mHeight = resolveSize(mHeight, heightMeasureSpec);
        Log.d(TAG, "after width,height=" + (mWidth + "," + mHeight));
        setMeasuredDimension(mWidth, mHeight);
    }

    @Override
    protected void onDraw(Canvas canvas) {
        if (mDrawable == null) {
            return;
        }
        canvas.drawBitmap(drawableToBitmap(), getLeft(), getTop(), mPaint);
    }

    public Bitmap drawableToBitmap() {
        if (mBitmap != null) {
            return mBitmap;
        }
        Bitmap.Config config = mDrawable.getOpacity() != PixelFormat.OPAQUE ? Bitmap.Config.ARGB_8888 : Bitmap.Config.RGB_565;
        Bitmap bitmap = Bitmap.createBitmap(mWidth, mHeight, config);
        Canvas canvas = new Canvas(bitmap);
        mDrawable.setBounds(0, 0, mWidth, mHeight);
        mDrawable.draw(canvas);
        return bitmap;
    }
```

### View 类中的 resolveSize 及 resolveSizeAndState 方法
```
public static int resolveSizeAndState(int size, int measureSpec, int childMeasuredState) {
    final int specMode = MeasureSpec.getMode(measureSpec);
    final int specSize = MeasureSpec.getSize(measureSpec);
    final int result;
    switch (specMode) {
        case MeasureSpec.AT_MOST:
            if (specSize < size) {
                result = specSize | MEASURED_STATE_TOO_SMALL;
            } else {
                result = size;
            }
            break;
        case MeasureSpec.EXACTLY:
            result = specSize;
            break;
        case MeasureSpec.UNSPECIFIED:
        default:
            result = size;
    }
    return result | (childMeasuredState & MEASURED_STATE_MASK);
}
```

---